In [1]:
import h5py
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Resizing, Dropout, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Input, Reshape, Conv2DTranspose, Cropping2D

from keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from tools import define_cnn, extract_features, build_model, predict_with_model, set_seed, build_model_and_predict
import warnings
warnings.filterwarnings('ignore')

c:\Users\jakub\anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Normal CNN 10-dim vector

In [2]:
# load clean gdp data, keep only year, region and real_gdp columns
ukraine = pd.read_csv("data/tabular_data_ukraine.csv")

# delete Kyiv and Kyiv_Oblast
# ukraine = ukraine[ukraine["region"] != "Kyiv"]
# ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast"]
ukraine = ukraine[ukraine["region"] != "Kyiv_Oblast_City"]

# get the data for 2021, 2022 and before 2022
ukraine_2022 = ukraine[ukraine["year"] == 2022]
ukraine = ukraine[ukraine["year"].astype(int) < 2022]
ukraine_2021 = ukraine[ukraine["year"] == 2021]
ukraine_2021.reset_index(drop=True, inplace=True)
ukraine_2022.reset_index(drop=True, inplace=True)
ukraine.reset_index(drop=True, inplace=True)

# Initialise a three dimensional array to store the images with the shape (number of images, height, width, channels)
X = np.zeros((len(ukraine), 765, 1076, 1))
y = np.zeros(len(ukraine))

# load the images
for i in range(len(ukraine)):

    # get year, region, and gdp
    year = ukraine["year"][i]
    region = ukraine["region"][i]
    gdp_value = ukraine["real_gdp"][i]

    # load the image
    file_name = f"{year}_{region}_hq.h5"
    file_path = f"data/annual_region_images/{file_name}"
    
    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]

    # add the values
    y[i] = gdp_value
    X[i, :, :, 0] = allangle_snow_free

# normalise the images and gdp data
maximum_x = X.max()
X = X / maximum_x

maximum_y = y.max()
y = y / maximum_y

# get indices for observations with  year = 2021: this is the test set
test_indices = np.where(ukraine["year"].astype(int) == 2021)[0]
train_indices = np.where(ukraine["year"].astype(int) != 2021)[0]

# get the train and test sets
X_train, y_train, X_test, y_test = X[train_indices], y[train_indices], X[test_indices], y[test_indices]

# get the prediction data
X_pred = np.zeros((len(ukraine_2022), 765, 1076, 1))
for i in range(len(ukraine_2022)):

    year = ukraine_2022["year"][i]
    region = ukraine_2022["region"][i]

    file_name = f"{year}_{region}.h5"
    file_path = f"data/annual_region_images/{file_name}"

    with h5py.File(file_path, 'r') as annual_region:
        allangle_snow_free = annual_region["AllAngle_Composite_Snow_Free"][:]
    
    X_pred[i, :, :, 0] = allangle_snow_free

X_pred = X_pred / maximum_x

In [3]:
# define the grid of parameters: network architecture and number of features
param_grid_cnn = {
    'n_features': [4],
    'n_conv': [2],
    'n_dense': [4],
}

# df to store the results
cnn_results = pd.DataFrame(columns = ["n_features", "n_conv", "n_dense", "mae", "mpe", "gdp_change"])

# train first stage, neural network
for n_features in param_grid_cnn['n_features']:
    for n_conv in param_grid_cnn['n_conv']:
        for n_dense in param_grid_cnn['n_dense']:
            set_seed(0)
            model = define_cnn(n_features = n_features, n_conv = n_conv, n_dense = n_dense)
            model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])
            model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

            # extract features
            ukraine_stage_2, ukraine_2021, ukraine_2022 = extract_features(model, ukraine, ukraine_2021, ukraine_2022, X_train, X_pred, X_test, n_features)
            del model

            # train second stage, xgboost
            train_data = ukraine_stage_2
            test_data = ukraine_2021
            prediction_data = ukraine_2022
            pre_war_data = pd.concat([train_data, test_data])
            selected_columns =  ["feature_" + str(i) for i in range(1, n_features+1)]

            # param_grid_xgb = {
            #     'eta': [0.01, 0.1, 0.2, 0.3],
            #     'gamma': [10, 20, 50, 100],
            #     'max_depth': [4, 6, 8, 10],
            #     'min_child_weight': [3, 4, 5, 6],
            #     'random_state': [0] 
            # }

            # param_grid_xgb = {
            #     'eta': [0.01, 0.1, 0.2, 0.3],
            #     'gamma': [10, 20, 50, 100],
            #     'max_depth': [4, 6, 8, 10],
            #     'min_child_weight': [3, 4, 5, 6],
            #     'random_state': [0] 
            # }

            param_grid_rf = {
                    'n_estimators': [50, 100, 200, 300],
                    'max_depth': [5, 10, 15, 20],  # Maximum depth of the tree
                    'min_samples_split': [2, 4, 6, 8],  # Minimum number of samples required to split an internal node
                    'min_samples_leaf': [2, 4, 6, 8],  # Minimum number of samples required to be at a leaf node
                    'random_state': [0],  # Ensures reproducibility
            }

            # mae, mpe, _, pred = build_model(train_data, test_data, selected_columns, model_type, param_grid_xgb, log_transform, scale)
            # gdp_change, best_params = predict_with_model(pre_war_data, prediction_data, selected_columns, model_type, param_grid_xgb, log_transform, scale)

            # mae, mpe, gdp_change, gdp_predictions = build_model_and_predict(pre_war_data, prediction_data, selected_columns, "xgboost", param_grid_xgb, log_transform = False, scale = False, total_metrics = True)
            mae, mpe, gdp_change, gdp_predictions = build_model_and_predict(pre_war_data, prediction_data, selected_columns, "random_forest", param_grid_rf, log_transform = False, scale = False, total_metrics = True)

            # add the results to the dataframe
            new_row = pd.DataFrame([{
                "n_features": n_features, 
                "n_conv": n_conv, 
                "n_dense": n_dense, 
                "mae": mae, 
                "mpe": mpe, 
                "gdp_change": gdp_change
            }])
            
            cnn_results = pd.concat([cnn_results, new_row], ignore_index=True)

            print("Results for n_features = ", n_features, "n_conv = ", n_conv, "n_dense = ", n_dense)



Epoch 1/50


3/3 [==============================] - 4s 845ms/step - loss: 0.0249 - mae: 0.1231 - val_loss: 0.1284 - val_mae: 0.2826
Epoch 2/50
3/3 [==============================] - 1s 485ms/step - loss: 0.0240 - mae: 0.1164 - val_loss: 0.1422 - val_mae: 0.1936
Epoch 3/50
3/3 [==============================] - 1s 478ms/step - loss: 0.0145 - mae: 0.0818 - val_loss: 0.1302 - val_mae: 0.2027
Epoch 4/50
3/3 [==============================] - 1s 488ms/step - loss: 0.0110 - mae: 0.0763 - val_loss: 0.1211 - val_mae: 0.2221
Epoch 5/50
3/3 [==============================] - 1s 466ms/step - loss: 0.0099 - mae: 0.0782 - val_loss: 0.1272 - val_mae: 0.2012
Epoch 6/50
3/3 [==============================] - 1s 481ms/step - loss: 0.0097 - mae: 0.0696 - val_loss: 0.1202 - val_mae: 0.2259
Epoch 7/50
3/3 [==============================] - 1s 479ms/step - loss: 0.0097 - mae: 0.0780 - val_loss: 0.1292 - val_mae: 0.1961
Epoch 8/50
3/3 [==============================] - 1s 478ms/step - loss: 0.0095 - mae: 

In [5]:
# add the predictions to the dataframe, save
ukraine_2022["gdp_prediction"] = gdp_predictions
ukraine_2022 = ukraine_2022[["region", "year", "gdp_prediction"]]
ukraine_2022.to_csv("data/gdp_predictions_ukraine.csv", index=False)

In [4]:
print(cnn_results)

  n_features n_conv n_dense          mae       mpe  gdp_change
0          4      2       4  3007.128729  6.047283  -29.381124
